In [1]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [2]:
import numpy
a = numpy.random.randn(4,4)

In [3]:
a = a.astype(numpy.float32)

In [4]:
a_gpu = cuda.mem_alloc(a.nbytes)

In [5]:
cuda.memcpy_htod(a_gpu, a)


In [6]:
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

In [7]:
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

In [8]:
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)

In [9]:
print(a_doubled)
print(a)

[[-1.5116448   1.5069792  -0.3993524   3.6708593 ]
 [ 0.3611594  -0.29357162  1.0491561   1.3986686 ]
 [-2.106901   -4.0572634   3.4508677   0.898142  ]
 [-0.6335153  -2.954916   -0.35193285 -2.8832653 ]]
[[-0.7558224   0.7534896  -0.1996762   1.8354297 ]
 [ 0.1805797  -0.14678581  0.52457803  0.6993343 ]
 [-1.0534505  -2.0286317   1.7254338   0.449071  ]
 [-0.31675765 -1.477458   -0.17596643 -1.4416326 ]]
